# Data-Driven Stock Analysis: Data Preprocessing Notebook
## Nifty 50 Stock Market Analysis - Data Extraction & Transformation
---

### 1. Import Required Libraries

In [16]:
import pandas as pd
import numpy as np
import yaml
import os
import warnings
from pathlib import Path
from datetime import datetime
import json

warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


### 2. Define Data Extraction Function from YAML Files

In [17]:
from pathlib import Path
import yaml

def load_yaml_data(file_path):
    """Load YAML file safely"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return yaml.safe_load(file)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None


def extract_stock_data_from_yaml(data_folder_path):
    """
    Extract stock data from YAML files organized by month and date
    Expected structure:
        data/
          2023-10/
            20231003.yaml   # contains list of dicts with keys: Ticker, open, high, ...
    """
    all_stock_data = {}

    data_path = Path(data_folder_path)
    if not data_path.exists():
        print(f"Data folder not found at {data_folder_path}")
        return all_stock_data

    for month_folder in sorted(data_path.iterdir()):
        if month_folder.is_dir():
            print(f"Processing month: {month_folder.name}")

            for yaml_file in sorted(month_folder.glob('*.yaml')):
                yaml_data = load_yaml_data(yaml_file)

                if not yaml_data:
                    continue

                # yaml_data is expected to be a list of records
                for record in yaml_data:
                    # handle both 'Ticker' and 'Symbol' just in case
                    symbol = record.get('Ticker') or record.get('Symbol')
                    if symbol is None:
                        continue

                    if symbol not in all_stock_data:
                        all_stock_data[symbol] = []

                    # parse fields safely
                    row = {
                        "Date": record.get("date") or yaml_file.stem,  # use date if present
                        "Symbol": symbol,
                        "Open": float(record.get("open", 0)),
                        "High": float(record.get("high", 0)),
                        "Low": float(record.get("low", 0)),
                        "Close": float(record.get("close", 0)),
                        "Volume": int(record.get("volume", 0)),
                        "Month": record.get("month"),
                    }
                    all_stock_data[symbol].append(row)

    return all_stock_data


### 3. Extract Data from YAML Folder

In [18]:
DATA_FOLDER = r"D:\Guvi_projects\Stock_market-analysis\data"

print("Extracting stock data from YAML files...")
stock_data_raw = extract_stock_data_from_yaml(DATA_FOLDER)

print(f"\n✓ Extracted data for {len(stock_data_raw)} stocks")
print(f"Sample symbols: {list(stock_data_raw.keys())[:5]}")


Extracting stock data from YAML files...
Processing month: 2023-10
Processing month: 2023-11
Processing month: 2023-12
Processing month: 2024-01
Processing month: 2024-02
Processing month: 2024-03
Processing month: 2024-04
Processing month: 2024-05
Processing month: 2024-06
Processing month: 2024-07
Processing month: 2024-08
Processing month: 2024-09
Processing month: 2024-10
Processing month: 2024-11

✓ Extracted data for 50 stocks
Sample symbols: ['SBIN', 'BAJFINANCE', 'TITAN', 'ITC', 'TCS']


### 4. Convert Raw Data to DataFrames and Clean

In [19]:
# 4. Convert Raw Data to DataFrames and Clean

def create_and_clean_dataframe(symbol_data):
    """
    Create DataFrame from symbol data and perform cleaning.

    Expects each item in symbol_data to have keys:
    Date, Symbol, Open, High, Low, Close, Volume, (optional) Month
    """
    df = pd.DataFrame(symbol_data)

    if df.empty:
        return df

    # 1) Convert Date column (e.g. '2023-10-03 05:30:00') to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # 2) Drop rows with invalid dates
    df = df.dropna(subset=['Date'])

    if df.empty:
        return df

    # 3) Sort by Date
    df = df.sort_values('Date').reset_index(drop=True)

    # 4) Ensure numeric types for OHLCV
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 5) Drop rows with missing or non-positive OHLC
    mask = (
        (df['Open']  > 0) &
        (df['High']  > 0) &
        (df['Low']   > 0) &
        (df['Close'] > 0)
    )
    df = df[mask].dropna(subset=['Open', 'High', 'Low', 'Close'])

    if df.empty:
        return df

    # 6) Calculate daily returns (percentage) and price change
    df = df.sort_values('Date').reset_index(drop=True)
    df['Daily_Return'] = df['Close'].pct_change() * 100.0
    df['Price_Change'] = df['Close'] - df['Open']

    return df


# Build cleaned DataFrames for each symbol from stock_data_raw
stock_dataframes = {}

print(f"Symbols in raw data: {len(stock_data_raw)}")
for symbol, data in stock_data_raw.items():
    df = create_and_clean_dataframe(data)
    if not df.empty:
        stock_dataframes[symbol] = df

print(f"✓ Created and cleaned {len(stock_dataframes)} stock dataframes")

# Safely show a sample if any exist
if stock_dataframes:
    first_symbol = list(stock_dataframes.keys())[0]
    print(f"\nSample data from {first_symbol}:")
    print(stock_dataframes[first_symbol].head())
else:
    print("\n⚠️ No valid stock dataframes were created. Check date parsing and OHLC filters.")


Symbols in raw data: 50
✓ Created and cleaned 50 stock dataframes

Sample data from SBIN:
                 Date Symbol   Open    High     Low   Close    Volume  \
0 2023-10-03 05:30:00   SBIN  596.6  604.90  589.60  602.95  15322196   
1 2023-10-04 05:30:00   SBIN  600.0  600.45  584.45  586.25  24914612   
2 2023-10-05 05:30:00   SBIN  590.0  594.35  587.10  592.15  13248028   
3 2023-10-06 05:30:00   SBIN  593.4  598.95  592.20  594.25   8216780   
4 2023-10-09 05:30:00   SBIN  588.0  589.00  581.55  585.10   9189597   

     Month  Daily_Return  Price_Change  
0  2023-10           NaN          6.35  
1  2023-10     -2.769716        -13.75  
2  2023-10      1.006397          2.15  
3  2023-10      0.354640          0.85  
4  2023-10     -1.539756         -2.90  


### 5. Create Master Consolidated DataFrame

In [20]:
# Combine all stock data into master dataframe
master_df_list = []

for symbol, df in stock_dataframes.items():
    master_df_list.append(df)

master_df = pd.concat(master_df_list, ignore_index=True)
master_df = master_df.sort_values(['Symbol', 'Date']).reset_index(drop=True)

print(f"✓ Master DataFrame created")
print(f"Shape: {master_df.shape}")
print(f"\nColumn Names: {master_df.columns.tolist()}")
print(f"\nData Info:")
print(master_df.info())
print(f"\nFirst few rows:")
print(master_df.head())

✓ Master DataFrame created
Shape: (14200, 10)

Column Names: ['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume', 'Month', 'Daily_Return', 'Price_Change']

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14200 entries, 0 to 14199
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          14200 non-null  datetime64[ns]
 1   Symbol        14200 non-null  object        
 2   Open          14200 non-null  float64       
 3   High          14200 non-null  float64       
 4   Low           14200 non-null  float64       
 5   Close         14200 non-null  float64       
 6   Volume        14200 non-null  int64         
 7   Month         14200 non-null  object        
 8   Daily_Return  14150 non-null  float64       
 9   Price_Change  14200 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 1.1+ MB
None

First few rows:
             

### 6. Load Sector Data from CSV

In [21]:
# Load sector classification data
sector_df = pd.read_csv('Sector_data - Sheet1.csv')

print("✓ Sector data loaded")
print(f"\nSector DataFrame shape: {sector_df.shape}")
print(sector_df.head(10))
print(f"\nUnique sectors: {sector_df['sector'].nunique()}")
print(f"\nSector distribution:")
print(sector_df['sector'].value_counts())

✓ Sector data loaded

Sector DataFrame shape: (50, 3)
              COMPANY         sector                         Symbol
0   ADANI ENTERPRISES  MISCELLANEOUS  ADANI ENTERPRISES: ADANIGREEN
1   ADANI PORTS & SEZ  MISCELLANEOUS  ADANI PORTS & SEZ: ADANIPORTS
2    APOLLO HOSPITALS  MISCELLANEOUS   APOLLO HOSPITALS: APOLLOHOSP
3        ASIAN PAINTS         PAINTS       ASIAN PAINTS: ASIANPAINT
4           AXIS BANK        BANKING            AXIS BANK: AXISBANK
5          BAJAJ AUTO    AUTOMOBILES         BAJAJ AUTO: BAJAJ-AUTO
6       BAJAJ FINANCE        FINANCE      BAJAJ FINANCE: BAJFINANCE
7       BAJAJ FINSERV        FINANCE      BAJAJ FINSERV: BAJAJFINSV
8  BHARAT ELECTRONICS        DEFENCE        BHARAT ELECTRONICS: BEL
9       BHARTI AIRTEL        TELECOM          BHARTI AIRTEL: AIRTEL

Unique sectors: 21

Sector distribution:
sector
BANKING            6
AUTOMOBILES        6
SOFTWARE           5
ENERGY             4
MISCELLANEOUS      3
PHARMACEUTICALS    3
FINANCE            3
RE

### 7. Merge Master Data with Sector Information

In [22]:
# Extract symbol mapping from sector data
symbol_mapping = {}
for idx, row in sector_df.iterrows():
    symbol_part = row['Symbol'].split(':')[-1].strip()  # Extract symbol after colon
    symbol_mapping[symbol_part] = row['sector']

# Create sector mapping series
sector_mapping_series = pd.Series(symbol_mapping)

# Merge sector info with master dataframe
master_df['Sector'] = master_df['Symbol'].map(symbol_mapping)

# Handle missing sectors
if master_df['Sector'].isna().sum() > 0:
    print(f"Warning: {master_df['Sector'].isna().sum()} rows have missing sector information")
    master_df = master_df.dropna(subset=['Sector'])

print("✓ Sector information merged successfully")
print(f"\nSample merged data:")
print(master_df[['Date', 'Symbol', 'Close', 'Volume', 'Sector']].head(10))

✓ Sector information merged successfully

Sample merged data:
                   Date      Symbol   Close   Volume         Sector
284 2023-10-03 05:30:00  ADANIPORTS  831.40  2453090  MISCELLANEOUS
285 2023-10-04 05:30:00  ADANIPORTS  824.60  4096647  MISCELLANEOUS
286 2023-10-05 05:30:00  ADANIPORTS  825.20  2083505  MISCELLANEOUS
287 2023-10-06 05:30:00  ADANIPORTS  830.75  1877058  MISCELLANEOUS
288 2023-10-09 05:30:00  ADANIPORTS  790.05  5994282  MISCELLANEOUS
289 2023-10-10 05:30:00  ADANIPORTS  819.50  7591327  MISCELLANEOUS
290 2023-10-11 05:30:00  ADANIPORTS  814.85  3281387  MISCELLANEOUS
291 2023-10-12 05:30:00  ADANIPORTS  814.95  2563606  MISCELLANEOUS
292 2023-10-13 05:30:00  ADANIPORTS  813.75  3801000  MISCELLANEOUS
293 2023-10-16 05:30:00  ADANIPORTS  805.65  2664794  MISCELLANEOUS


### 8. Calculate Key Metrics

In [23]:
# Calculate yearly metrics for each stock
yearly_metrics = []

for symbol in master_df['Symbol'].unique():
    symbol_data = master_df[master_df['Symbol'] == symbol].sort_values('Date')
    
    if len(symbol_data) > 0:
        first_close = symbol_data['Close'].iloc[0]
        last_close = symbol_data['Close'].iloc[-1]
        
        yearly_return = ((last_close - first_close) / first_close) * 100
        volatility = symbol_data['Daily_Return'].std()
        avg_volume = symbol_data['Volume'].mean()
        avg_price = symbol_data['Close'].mean()
        max_price = symbol_data['Close'].max()
        min_price = symbol_data['Close'].min()
        
        sector = symbol_data['Sector'].iloc[0] if 'Sector' in symbol_data.columns else 'Unknown'
        
        yearly_metrics.append({
            'Symbol': symbol,
            'Sector': sector,
            'Yearly_Return': yearly_return,
            'Volatility': volatility,
            'Avg_Price': avg_price,
            'Max_Price': max_price,
            'Min_Price': min_price,
            'Avg_Volume': avg_volume,
            'Start_Price': first_close,
            'End_Price': last_close,
            'Price_Change': last_close - first_close
        })

metrics_df = pd.DataFrame(yearly_metrics).sort_values('Yearly_Return', ascending=False)

print("✓ Yearly metrics calculated")
print(f"\nTop 10 Green Stocks (Gainers):")
print(metrics_df[['Symbol', 'Sector', 'Yearly_Return', 'Volatility']].head(10))
print(f"\nTop 10 Red Stocks (Losers):")
print(metrics_df[['Symbol', 'Sector', 'Yearly_Return', 'Volatility']].tail(10))

✓ Yearly metrics calculated

Top 10 Green Stocks (Gainers):
        Symbol           Sector  Yearly_Return  Volatility
43       TRENT        RETAILING     223.092613    2.307380
7          BEL          DEFENCE     101.760057    2.328339
27         M&M      AUTOMOBILES      95.976974    1.921277
4   BAJAJ-AUTO      AUTOMOBILES      89.011153    1.760937
32   POWERGRID            POWER      68.854924    1.867144
8         BPCL           ENERGY      67.477150    2.206885
17  HEROMOTOCO      AUTOMOBILES      58.976655    1.656324
37   SUNPHARMA  PHARMACEUTICALS      57.282404    1.173282
14     HCLTECH         SOFTWARE      53.257447    1.429465
30        NTPC            POWER      51.513267    1.947501

Top 10 Red Stocks (Losers):
        Symbol          Sector  Yearly_Return  Volatility
23         ITC  FOOD & TOBACCO       7.936327    1.194370
28      MARUTI     AUTOMOBILES       6.926712    1.371249
42       TITAN       RETAILING       3.518185    1.413589
5   BAJAJFINSV         FINANCE

In [24]:
# Diagnostic: Check which stocks were filtered out
print("="*60)
print("DIAGNOSTIC: Stock Data Flow Analysis")
print("="*60)

stocks_in_raw = set(stock_data_raw.keys())
stocks_in_dataframes = set(stock_dataframes.keys())
stocks_with_metrics = set(metrics_df['Symbol'].unique())

filtered_in_cleaning = stocks_in_raw - stocks_in_dataframes
filtered_in_metrics = stocks_in_dataframes - stocks_with_metrics

print(f"\n1. Stocks in raw data: {len(stocks_in_raw)}")
print(f"2. Stocks in dataframes (after cleaning): {len(stocks_in_dataframes)}")
print(f"3. Stocks in metrics: {len(stocks_with_metrics)}")

if filtered_in_cleaning:
    print(f"\n❌ Stocks FILTERED OUT during data cleaning ({len(filtered_in_cleaning)}):")
    for symbol in sorted(filtered_in_cleaning):
        print(f"   - {symbol}")
        if symbol in stock_data_raw:
            raw_count = len(stock_data_raw[symbol])
            print(f"     (had {raw_count} raw records)")

if filtered_in_metrics:
    print(f"\n❌ Stocks FILTERED OUT during metrics calculation ({len(filtered_in_metrics)}):")
    for symbol in sorted(filtered_in_metrics):
        print(f"   - {symbol}")
        # Check why they were filtered
        if symbol in master_df['Symbol'].values:
            sector_value = master_df[master_df['Symbol'] == symbol]['Sector'].unique()[0]
            print(f"     Reason: Sector = {sector_value} (missing sector mapping)")
        else:
            print(f"     Reason: Not found in master_df")

print("\n" + "="*60)
print("ROOT CAUSE:")
print("="*60)
print("The 4 missing stocks are NOT in the sector CSV file.")
print("This means the symbol mapping failed during the merge step.")
print("\nLet's check the sector CSV:")
print(f"Symbols in sector CSV: {sorted(symbol_mapping.keys())[:10]}... ({len(symbol_mapping)} total)")
print(f"\nMissing from sector mapping:")
for symbol in sorted(filtered_in_metrics):
    print(f"  - {symbol}")
print("="*60)

DIAGNOSTIC: Stock Data Flow Analysis

1. Stocks in raw data: 50
2. Stocks in dataframes (after cleaning): 50
3. Stocks in metrics: 46

❌ Stocks FILTERED OUT during metrics calculation (4):
   - ADANIENT
     Reason: Not found in master_df
   - BHARTIARTL
     Reason: Not found in master_df
   - BRITANNIA
     Reason: Not found in master_df
   - TATACONSUM
     Reason: Not found in master_df

ROOT CAUSE:
The 4 missing stocks are NOT in the sector CSV file.
This means the symbol mapping failed during the merge step.

Let's check the sector CSV:
Symbols in sector CSV: ['ADANIGREEN', 'ADANIPORTS', 'AIRTEL', 'APOLLOHOSP', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BEL']... (50 total)

Missing from sector mapping:
  - ADANIENT
  - BHARTIARTL
  - BRITANNIA
  - TATACONSUM


### 9. Generate Market Summary

In [25]:
# Market summary statistics
green_stocks = (metrics_df['Yearly_Return'] > 0).sum()
red_stocks = (metrics_df['Yearly_Return'] < 0).sum()
total_stocks = len(metrics_df)

avg_return = metrics_df['Yearly_Return'].mean()
avg_price = master_df['Close'].mean()
avg_volume = master_df['Volume'].mean()

market_summary = {
    'Total_Stocks': total_stocks,
    'Green_Stocks': green_stocks,
    'Red_Stocks': red_stocks,
    'Green_Percentage': (green_stocks / total_stocks) * 100,
    'Red_Percentage': (red_stocks / total_stocks) * 100,
    'Avg_Return': avg_return,
    'Avg_Price': avg_price,
    'Avg_Volume': avg_volume,
    'Market_Return': metrics_df['Yearly_Return'].sum()
}

print("✓ Market Summary Generated")
print(f"\n{'='*50}")
print("MARKET OVERVIEW")
print(f"{'='*50}")
for key, value in market_summary.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")
print(f"{'='*50}")

✓ Market Summary Generated

MARKET OVERVIEW
Total_Stocks: 46
Green_Stocks: 42
Red_Stocks: 4
Green_Percentage: 91.30
Red_Percentage: 8.70
Avg_Return: 33.95
Avg_Price: 2432.51
Avg_Volume: 7197210.97
Market_Return: 1561.77


### 10. Save Processed Data to CSV Files

In [26]:
# Create output directory
output_dir = './processed_data'
os.makedirs(output_dir, exist_ok=True)

# Save individual stock CSVs
for symbol in master_df['Symbol'].unique():
    symbol_data = master_df[master_df['Symbol'] == symbol]
    filename = f"{output_dir}/{symbol}_historical_data.csv"
    symbol_data.to_csv(filename, index=False)

# Save consolidated data
master_df.to_csv(f"{output_dir}/consolidated_stock_data.csv", index=False)

# Save metrics
metrics_df.to_csv(f"{output_dir}/yearly_metrics.csv", index=False)

# --------- SAFE MARKET SUMMARY (CAST TO PYTHON TYPES) ---------
# Rebuild / cast market_summary to native Python types
green_stocks = int((metrics_df['Yearly_Return'] > 0).sum())
red_stocks   = int((metrics_df['Yearly_Return'] < 0).sum())
total_stocks = int(len(metrics_df))

avg_return = float(metrics_df['Yearly_Return'].mean())
avg_price  = float(master_df['Close'].mean())
avg_volume = float(master_df['Volume'].mean())
market_return = float(metrics_df['Yearly_Return'].sum())

market_summary = {
    'Total_Stocks': total_stocks,
    'Green_Stocks': green_stocks,
    'Red_Stocks': red_stocks,
    'Green_Percentage': float((green_stocks / total_stocks) * 100) if total_stocks else 0.0,
    'Red_Percentage': float((red_stocks / total_stocks) * 100) if total_stocks else 0.0,
    'Avg_Return': avg_return,
    'Avg_Price': avg_price,
    'Avg_Volume': avg_volume,
    'Market_Return': market_return,
}

# Save market summary as JSON
with open(f"{output_dir}/market_summary.json", 'w') as f:
    json.dump(market_summary, f, indent=4)

print(f"✓ Data saved successfully to '{output_dir}' directory")
print("\nFiles created:")
print(f"  - {len(master_df['Symbol'].unique())} individual stock CSV files")
print("  - consolidated_stock_data.csv (Master data)")
print("  - yearly_metrics.csv")
print("  - market_summary.json")


✓ Data saved successfully to './processed_data' directory

Files created:
  - 46 individual stock CSV files
  - consolidated_stock_data.csv (Master data)
  - yearly_metrics.csv
  - market_summary.json


### 11. Calculate Additional Metrics for Visualization

In [27]:
# Calculate cumulative returns for top performers
top_5_gainers = metrics_df.head(5)['Symbol'].tolist()

cumulative_returns_data = {}

for symbol in master_df['Symbol'].unique():
    symbol_data = master_df[master_df['Symbol'] == symbol].sort_values('Date')
    
    if len(symbol_data) > 0:
        # Calculate cumulative return
        symbol_data['Cumulative_Return'] = (1 + symbol_data['Daily_Return'] / 100).cumprod() - 1
        cumulative_returns_data[symbol] = symbol_data[['Date', 'Symbol', 'Close', 'Cumulative_Return']]

cumulative_df = pd.concat(cumulative_returns_data.values(), ignore_index=True)

print("✓ Cumulative returns calculated")
print(f"\nCumulative returns for top 5 gainers:")
for symbol in top_5_gainers:
    data = cumulative_df[cumulative_df['Symbol'] == symbol]
    if not data.empty:
        final_return = data['Cumulative_Return'].iloc[-1] * 100
        print(f"{symbol}: {final_return:.2f}%")

✓ Cumulative returns calculated

Cumulative returns for top 5 gainers:
TRENT: 223.09%
BEL: 101.76%
M&M: 95.98%
BAJAJ-AUTO: 89.01%
POWERGRID: 68.85%


### 12. Sector-wise Analysis

In [28]:
# Sector-wise performance analysis
sector_performance = metrics_df.groupby('Sector').agg({
    'Yearly_Return': ['mean', 'count'],
    'Volatility': 'mean',
    'Avg_Volume': 'mean',
    'Avg_Price': 'mean'
}).round(2)

sector_performance.columns = ['Avg_Return', 'Stock_Count', 'Avg_Volatility', 'Avg_Price', 'Avg_Volume']
sector_performance = sector_performance.sort_values('Avg_Return', ascending=False)

print("✓ Sector-wise analysis completed")
print(f"\nSector Performance:")
print(sector_performance)

# Save sector performance
sector_performance.to_csv(f"{output_dir}/sector_performance.csv")

✓ Sector-wise analysis completed

Sector Performance:
                 Avg_Return  Stock_Count  Avg_Volatility    Avg_Price  \
Sector                                                                  
RETAILING            113.31            2            1.86   1012349.25   
DEFENCE              101.76            1            2.33  28472446.58   
POWER                 60.18            2            1.91  17084515.97   
AUTOMOBILES           54.53            6            1.68   2936177.92   
MINING                41.85            1            2.14  11918318.84   
SOFTWARE              38.28            5            1.51   4404313.15   
CEMENT                36.97            1            1.43    352559.78   
ENERGY                36.56            3            1.93  15811123.87   
MISCELLANEOUS         36.10            2            2.01   2623519.89   
ALUMINIUM             35.87            1            1.96   6997897.32   
TEXTILES              35.78            1            1.45    800653.35 

### 13. Correlation Analysis

In [29]:
# Create pivot table for correlation analysis
pivot_close = master_df.pivot_table(
    index='Date',
    columns='Symbol',
    values='Close'
)

# Calculate correlation matrix
correlation_matrix = pivot_close.corr().round(3)

print("✓ Correlation matrix calculated")
print(f"\nCorrelation matrix shape: {correlation_matrix.shape}")
print(f"\nSample correlations:")
print(correlation_matrix.iloc[:5, :5])

# Save correlation matrix
correlation_matrix.to_csv(f"{output_dir}/correlation_matrix.csv")

✓ Correlation matrix calculated

Correlation matrix shape: (46, 46)

Sample correlations:
Symbol      ADANIPORTS  APOLLOHOSP  ASIANPAINT  AXISBANK  BAJAJ-AUTO
Symbol                                                              
ADANIPORTS       1.000       0.815      -0.247     0.770       0.888
APOLLOHOSP       0.815       1.000      -0.158     0.607       0.894
ASIANPAINT      -0.247      -0.158       1.000     0.001      -0.163
AXISBANK         0.770       0.607       0.001     1.000       0.743
BAJAJ-AUTO       0.888       0.894      -0.163     0.743       1.000


### 14. Monthly Performance Analysis

In [30]:
# Add month-year column
master_df['Month_Year'] = master_df['Date'].dt.to_period('M')

# Monthly performance for each stock
monthly_performance = []

for month in sorted(master_df['Month_Year'].unique()):
    month_data = master_df[master_df['Month_Year'] == month]
    
    for symbol in month_data['Symbol'].unique():
        symbol_month_data = month_data[month_data['Symbol'] == symbol]
        
        if len(symbol_month_data) > 0:
            first_price = symbol_month_data['Close'].iloc[0]
            last_price = symbol_month_data['Close'].iloc[-1]
            monthly_return = ((last_price - first_price) / first_price) * 100
            
            monthly_performance.append({
                'Month_Year': str(month),
                'Symbol': symbol,
                'Monthly_Return': monthly_return,
                'Avg_Price': symbol_month_data['Close'].mean(),
                'Volume': symbol_month_data['Volume'].sum()
            })

monthly_df = pd.DataFrame(monthly_performance)

print("✓ Monthly performance analysis completed")
print(f"\nMonthly performance data points: {len(monthly_df)}")
print(f"\nSample:")
print(monthly_df.head(10))

# Save monthly performance
monthly_df.to_csv(f"{output_dir}/monthly_performance.csv", index=False)

✓ Monthly performance analysis completed

Monthly performance data points: 644

Sample:
  Month_Year      Symbol  Monthly_Return  Avg_Price     Volume
0    2023-10  ADANIPORTS       -5.611018   801.1975   63920642
1    2023-10  APOLLOHOSP       -5.842018  4978.7800    7880043
2    2023-10  ASIANPAINT       -5.404424  3102.7800   18372958
3    2023-10    AXISBANK       -5.686566   996.1975  170452203
4    2023-10  BAJAJ-AUTO        5.932482  5178.2325   11508601
5    2023-10  BAJAJFINSV        0.544505  1606.1500   32829911
6    2023-10  BAJFINANCE       -5.961017  7857.2325   18942645
7    2023-10         BEL       -4.274425   135.9725  218706151
8    2023-10        BPCL        2.314272   172.3720  145737290
9    2023-10       CIPLA        1.454177  1175.8950   34195095


### 15. Data Export for Streamlit Application

In [31]:
# Prepare data export for Streamlit app
export_data = {
    'master_data': master_df,
    'metrics': metrics_df,
    'correlation_matrix': correlation_matrix,
    'monthly_performance': monthly_df,
    'market_summary': market_summary
}

# Save as pickle for efficient loading in Streamlit
import pickle

with open(f"{output_dir}/processed_data.pkl", 'wb') as f:
    pickle.dump(export_data, f)

print("✓ All data processed and exported successfully!")
print(f"\n{'='*50}")
print("PREPROCESSING SUMMARY")
print(f"{'='*50}")
print(f"Total stocks analyzed: {len(metrics_df)}")
print(f"Date range: {master_df['Date'].min()} to {master_df['Date'].max()}")
print(f"Total data points: {len(master_df)}")
print(f"Sectors covered: {metrics_df['Sector'].nunique()}")
print(f"\n✓ All files saved in '{output_dir}' directory")
print(f"✓ Ready for Streamlit application!")
print(f"{'='*50}")

✓ All data processed and exported successfully!

PREPROCESSING SUMMARY
Total stocks analyzed: 46
Date range: 2023-10-03 05:30:00 to 2024-11-22 05:30:00
Total data points: 13064
Sectors covered: 20

✓ All files saved in './processed_data' directory
✓ Ready for Streamlit application!


In [32]:
# SOLUTION: Add missing stocks with their sectors
print("="*60)
print("SOLUTION: Adding Missing Stocks Sectors")
print("="*60)

# Check what's in sector_df
print("\nAll symbols in sector CSV:")
print(sector_df[['Symbol', 'sector']].to_string())

print("\n\nSymbols in YAML data but missing from sector mapping:")
missing_stocks = []
for symbol in sorted(filtered_in_metrics):
    missing_stocks.append(symbol)
    print(f"  - {symbol}")

# Manual mapping for missing stocks (research these companies)
missing_sector_mapping = {
    'ADANIENT': 'MISCELLANEOUS',      # Adani Enterprises (parent company)
    'BHARTIARTL': 'TELECOM',           # Bharti Airtel
    'BRITANNIA': 'FOOD & TOBACCO',     # Britannia Industries
    'TATACONSUM': 'FOOD & TOBACCO'     # Tata Consumer Products
}

print(f"\n\nManually adding sectors for missing stocks:")
for symbol, sector in missing_sector_mapping.items():
    print(f"  {symbol} -> {sector}")
    
# Update the symbol_mapping dictionary
for symbol, sector in missing_sector_mapping.items():
    symbol_mapping[symbol] = sector

# Re-merge sector info with master_df
master_df['Sector'] = master_df['Symbol'].map(symbol_mapping)

# Check if all sectors are now mapped
unmapped = master_df[master_df['Sector'].isna()]['Symbol'].unique()
if len(unmapped) > 0:
    print(f"\n⚠️ Still unmapped symbols: {unmapped}")
else:
    print(f"\n✓ All stocks now have sectors!")

# Remove any rows with missing sectors
master_df = master_df.dropna(subset=['Sector'])

print(f"\nUpdated master_df size: {len(master_df)} rows")
print(f"Unique stocks now: {master_df['Symbol'].nunique()}")


SOLUTION: Adding Missing Stocks Sectors

All symbols in sector CSV:
                            Symbol           sector
0    ADANI ENTERPRISES: ADANIGREEN    MISCELLANEOUS
1    ADANI PORTS & SEZ: ADANIPORTS    MISCELLANEOUS
2     APOLLO HOSPITALS: APOLLOHOSP    MISCELLANEOUS
3         ASIAN PAINTS: ASIANPAINT           PAINTS
4              AXIS BANK: AXISBANK          BANKING
5           BAJAJ AUTO: BAJAJ-AUTO      AUTOMOBILES
6        BAJAJ FINANCE: BAJFINANCE          FINANCE
7        BAJAJ FINSERV: BAJAJFINSV          FINANCE
8          BHARAT ELECTRONICS: BEL          DEFENCE
9            BHARTI AIRTEL: AIRTEL          TELECOM
10                      BPCL: BPCL           ENERGY
11                    CIPLA: CIPLA  PHARMACEUTICALS
12           COAL INDIA: COALINDIA           MINING
13         DR. REDDYS LAB: DRREDDY  PHARMACEUTICALS
14        EICHER MOTORS: EICHERMOT      AUTOMOBILES
15                  GRASIM: GRASIM         TEXTILES
16       HCL TECHNOLOGIES: HCLTECH         SOFTW

In [33]:
# Add this at the end of your data processing script
import os

# Create export folder
os.makedirs('powerbi_data', exist_ok=True)

# 1. Master Data (Stock Prices)
master_df.to_csv('powerbi_data/master_prices.csv', index=False)

# 2. Metrics Summary (One row per stock)
metrics_df.to_csv('powerbi_data/stock_metrics.csv', index=False)

# 3. Monthly Performance
monthly_df.to_csv('powerbi_data/monthly_performance.csv', index=False)

# 4. Market Summary (Single row)
pd.DataFrame([market_summary]).to_csv('powerbi_data/market_summary.csv', index=False)

# 5. Correlation Matrix (Unpivoted for Power BI)
corr_unpivot = correlation_matrix.reset_index().melt(
    id_vars='index', 
    var_name='Stock2', 
    value_name='Correlation'
)
corr_unpivot.rename(columns={'index': 'Stock1'}, inplace=True)
corr_unpivot.to_csv('powerbi_data/correlations.csv', index=False)

print("✅ Data exported to 'powerbi_data' folder!")

KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['index']"

In [34]:
"""
Complete Data Export Script for Power BI - FIXED VERSION
Handles all data structure variations
"""

import pandas as pd
import numpy as np
import os
import pickle
from datetime import datetime

# Create export directory
export_path = 'powerbi_data'
os.makedirs(export_path, exist_ok=True)
print(f"📁 Export folder created: {export_path}")

# ============================================================================
# LOAD DATA
# ============================================================================

try:
    _ = master_df
    print("✅ Using existing data from memory")
except NameError:
    print("🔄 Loading data from pickle file...")
    try:
        with open('./processed_data/processed_data.pkl', 'rb') as f:
            data = pickle.load(f)
        
        master_df = data.get('master_data')
        metrics_df = data.get('metrics')
        correlation_matrix = data.get('correlation_matrix')
        monthly_df = data.get('monthly_performance')
        market_summary = data.get('market_summary')
        
        print("✅ Data loaded successfully")
    except FileNotFoundError:
        print("❌ Error: processed_data.pkl not found!")
        exit()

# ============================================================================
# CHECK AND PREPARE DATA
# ============================================================================

print("\n🔍 Preparing data...")

# Create monthly_df if missing
if 'monthly_df' not in locals() or monthly_df is None:
    print("⚠️  Creating monthly_df from master_df...")
    try:
        master_df['Date'] = pd.to_datetime(master_df['Date'])
        master_df['Month_Year'] = master_df['Date'].dt.to_period('M').astype(str)
        
        monthly_df = master_df.groupby(['Month_Year', 'Symbol']).agg({
            'Close': ['first', 'last']
        }).reset_index()
        
        monthly_df.columns = ['Month_Year', 'Symbol', 'Open', 'Close']
        monthly_df['Monthly_Return'] = ((monthly_df['Close'] - monthly_df['Open']) / monthly_df['Open']) * 100
        print("✅ monthly_df created")
    except Exception as e:
        print(f"❌ Error: {e}")
        monthly_df = pd.DataFrame(columns=['Month_Year', 'Symbol', 'Monthly_Return'])

# Create market_summary if missing
if 'market_summary' not in locals() or market_summary is None:
    print("⚠️  Creating market_summary...")
    try:
        total = len(metrics_df)
        green = len(metrics_df[metrics_df['Yearly_Return'] > 0])
        red = total - green
        
        market_summary = {
            'Total_Stocks': total,
            'Green_Stocks': green,
            'Red_Stocks': red,
            'Green_Percentage': (green/total)*100 if total > 0 else 0,
            'Red_Percentage': (red/total)*100 if total > 0 else 0,
            'Avg_Return': metrics_df['Yearly_Return'].mean()
        }
        print("✅ market_summary created")
    except:
        market_summary = {}

# ============================================================================
# EXPORT TO CSV - CORRECTED VERSION
# ============================================================================

print("\n📤 Exporting files...")

try:
    # 1. Master Prices
    master_df.to_csv(f'{export_path}/01_master_prices.csv', index=False)
    print(f"✅ 01_master_prices.csv ({len(master_df):,} rows)")
    
    # 2. Stock Metrics
    metrics_df.to_csv(f'{export_path}/02_stock_metrics.csv', index=False)
    print(f"✅ 02_stock_metrics.csv ({len(metrics_df):,} rows)")
    
    # 3. Monthly Performance
    monthly_df.to_csv(f'{export_path}/03_monthly_performance.csv', index=False)
    print(f"✅ 03_monthly_performance.csv ({len(monthly_df):,} rows)")
    
    # 4. Market Summary
    pd.DataFrame([market_summary]).to_csv(f'{export_path}/04_market_summary.csv', index=False)
    print(f"✅ 04_market_summary.csv (1 row)")
    
    # 5. Correlation Matrix - FIXED VERSION
    if correlation_matrix is not None:
        # Ensure index has a name
        if correlation_matrix.index.name is None:
            correlation_matrix.index.name = 'Stock1'
        
        # Reset index to make Stock1 a column
        corr_reset = correlation_matrix.reset_index()
        
        # Melt from wide to long format
        corr_unpivot = corr_reset.melt(
            id_vars=['Stock1'],           # Column name is 'Stock1', not 'index'
            var_name='Stock2', 
            value_name='Correlation'
        )
        
        # Remove self-correlations (optional, keeps file smaller)
        corr_unpivot = corr_unpivot[corr_unpivot['Stock1'] != corr_unpivot['Stock2']]
        
        corr_unpivot.to_csv(f'{export_path}/05_correlations.csv', index=False)
        print(f"✅ 05_correlations.csv ({len(corr_unpivot):,} rows)")
    
    # 6. Sector Summary
    if 'Sector' in metrics_df.columns:
        sector_summary = metrics_df.groupby('Sector').agg({
            'Yearly_Return': ['mean', 'count', 'sum'],
            'Volatility': 'mean',
            'Symbol': lambda x: ','.join(x)
        }).round(2)
        sector_summary.columns = ['Avg_Return', 'Stock_Count', 'Total_Return', 'Avg_Volatility', 'Stocks']
        sector_summary = sector_summary.reset_index()
        sector_summary.to_csv(f'{export_path}/06_sector_summary.csv', index=False)
        print(f"✅ 06_sector_summary.csv ({len(sector_summary)} rows)")
    
    # 7. Date Table
    if 'Date' in master_df.columns:
        dates_df = pd.DataFrame()
        dates_df['Date'] = pd.to_datetime(master_df['Date']).unique()
        dates_df = dates_df.sort_values('Date').drop_duplicates()
        dates_df['Year'] = pd.to_datetime(dates_df['Date']).dt.year
        dates_df['Month'] = pd.to_datetime(dates_df['Date']).dt.month
        dates_df['Month_Name'] = pd.to_datetime(dates_df['Date']).dt.month_name()
        dates_df['Quarter'] = pd.to_datetime(dates_df['Date']).dt.quarter
        dates_df['Month_Year'] = pd.to_datetime(dates_df['Date']).dt.to_period('M').astype(str)
        dates_df.to_csv(f'{export_path}/07_date_table.csv', index=False)
        print(f"✅ 07_date_table.csv ({len(dates_df):,} rows)")

    print(f"\n✨ EXPORT COMPLETE! Files saved to: '{export_path}/'")

except Exception as e:
    print(f"\n❌ Export error: {e}")
    import traceback
    traceback.print_exc()

📁 Export folder created: powerbi_data
✅ Using existing data from memory

🔍 Preparing data...

📤 Exporting files...
✅ 01_master_prices.csv (13,064 rows)
✅ 02_stock_metrics.csv (46 rows)
✅ 03_monthly_performance.csv (644 rows)
✅ 04_market_summary.csv (1 row)

❌ Export error: "The following id_vars or value_vars are not present in the DataFrame: ['Stock1']"


Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_9612\2994706649.py", line 118, in <module>
    corr_unpivot = corr_reset.melt(
        id_vars=['Stock1'],           # Column name is 'Stock1', not 'index'
        var_name='Stock2',
        value_name='Correlation'
    )
  File "d:\Guvi_projects\Stock_market-analysis\venv_stock\Lib\site-packages\pandas\core\frame.py", line 9969, in melt
    return melt(
           ~~~~^
        self,
        ^^^^^
    ...<5 lines>...
        ignore_index=ignore_index,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^
    ).__finalize__(self, method="melt")
    ^
  File "d:\Guvi_projects\Stock_market-analysis\venv_stock\Lib\site-packages\pandas\core\reshape\melt.py", line 74, in melt
    raise KeyError(
    ...<2 lines>...
    )
KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['Stock1']"


✅ Correlations: 2070 rows
Done! Check powerbi_data folder
